In [47]:
import sys
sys.path.append('/global/homes/s/sleak/.local/cori/3.6-anaconda-4.4/lib/python3.6/site-packages')
import rdflib

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import os
print(os.getcwd())

sys.path.append('/global/homes/s/sleak/Monitoring/Resilience/LogSet/src')
import LogsGraph    

# force re-read of LogsGraph.py in case I've edited it:
import importlib
importlib.reload(LogsGraph)

def dump(graph):
    #for s,o,p in graph:
    #    print ('{0} --- {1} --- {2}'.format(s,o,p))
    print(graph.serialize(format='n3').decode('ascii'))

import urllib.request
def parse_line_by_line(url, fmt):
    """ rdflib is really uninformative about the location of the error
        when it can't parse a file, so use this to parse line-by-line 
        and look for errors
        (hmm, does not work, fails on valid files)
    """
    f = urllib.request.urlopen(url)
    g = rdflib.ConjunctiveGraph()
    linecount=0
    for line in f.readlines():
        linecount += 1
        print("parsing line {0:d}: {1}".format(linecount,line))
        rdftext = line.strip()
        if not rdftext: continue
        try:
            g.parse(data=rdftext, format=fmt)
        except:
            print("failed to parse line {0:d} of {1}".format(linecount,url))
            print(line)
            print(g.serialize())
            raise
    print("finished parsing {0:s} successfully!".format(url))
    

# quick sanity check:
#parse_line_by_line('file:/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc-entities.ttl', 'turtle')
#g1 = rdflib.ConjunctiveGraph().parse("file:/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index.ttl", format='turtle')
#print("got this far...")
#print(g1.serialize(format='n3').decode('ascii'))

print("hello")

#graph = LogsGraph.construct('nersc-catalog.ttl', spider=True)
f1='file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/nersc.ttl'
f2='file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict.ttl'
graph = LogsGraph.construct(f1,f2,spider=True)

query = '''select ?id ?title where { ?id a dcat:Catalog .
                                     ?id dct:title ?title . }'''
print("\nCatalogs:")
for row in graph.query(query):
    print('{0}: {1}'.format(str(row[0]),str(row[1])))

query = '''select ?id ?desc where { ?id a logset:LogSet .
                                     ?id dct:description ?desc . }'''
print("\nLogSets:")
for row in graph.query(query):
    print('{0}: {1}'.format(str(row[0]),str(row[1])))

    
#query = 'select ?id ?val WHERE { ?id a ?val . }'
#for row in graph.query(query):
#    print('{0} --- {1}'.format(str(row[0]),str(row[1])))
#    #print(row)
#
#
#query = '''SELECT ?id ?what ?path
#WHERE {
#?id a logset:ConcreteLog . 
#?id dcat:downloadURL ?path .
#?id logset:isInstanceOf ?what .
#} '''
#
#print("concretelogs:")
#for row in graph.query(query):
#    print(row)
#
#query = '''SELECT ?id ?path
#WHERE {
#?id a logset:ConcreteLog .
#?id logset:isInstanceOf dict:console_logfile . 
#?id dcat:downloadURL ?path .
#} '''
#print("console logfiles:")
#for row in graph.query(query):
#    print('{0} --- {1}'.format(str(row[0]),str(row[1]))) 
#dump(graph)
# why am I not finding the console-log that is in the snl index?
# ah, typo in index.ttl, point dict prefix at vocab instead .. fixed now
# first incarnation will prob be rather fragile...
                
    

#[ns for ns in graph.namespaces()]

DEBUG:root:parsed has: set()
DEBUG:root:unparsed has: {'file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict.ttl', 'http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#', 'file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/nersc.ttl', 'http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#'}
DEBUG:root:looking for file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict.ttl
DEBUG:root:namespaces are now: [('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')), ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')), ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')), ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')), ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')), ('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#')), ('logset', rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datase

/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples
sys.version_info(major=3, minor=6, micro=3, releaselevel='final', serial=0)
hello

Catalogs:

LogSets:


In [48]:
#import rdflib.tools.rdf2dot
#stream = open('test.dot', 'w')
#rdflib.tools.rdf2dot.rdf2dot(graph, stream)
#stream.close()

[ns for ns in graph.namespaces()]
#dir(rdflib.namespace)

#print(rdflib.namespace._RDFNamespace)
#print(rdflib.namespace.RDFS)
#print (rdflib.namespace.namespaces())
#, RDFS, OWL, XSD, FOAF, SKOS, DOAP, DC, DCTERMS, VOID: print (rdflib.namespace.i)

[('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#')),
 ('logset',
  rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#')),
 ('dict',
  rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#')),
 ('',
  rdflib.term.URIRef('file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict#')),
 ('dct', rdflib.term.URIRef('http://purl.org/dc/terms/')),
 ('foaf', rdflib.term.URIRef('http://xmlns.com/foaf/0.1/')),
 ('adms', rdflib.term.URIRef('http://www.w3.org/ns/adms#')),
 ('default1',
  rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/re

In [49]:
dump(graph)
# find datasets, read them, and if they are LogSets, add them to the graph:

@prefix : <file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict#> .
@prefix adms: <http://www.w3.org/ns/adms#> .
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix default1: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#> .
@prefix default2: <file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/nersc#> .
@prefix default3: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#> .
@prefix dict: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix logset: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> 

In [50]:
for row in graph.query('''select ?id ?pred ?obj where { ?id a logset:LogSet . 
            ?id ?pred ?obj . }'''):
    #print ('{0} {1} {2}'.format(str(row[0]),str(row[1]),str(row[2])))
    #print ('{0} {1}'.format(str(row[1]),str(row[2])))
    print (str(row[0]))



In [53]:
print("regexs for common file patterns:")
query = '''select ?tag ?regex where {
    ?id a dict:FilenamePattern .
    ?id dict:tag ?tag .
    ?id dict:regex ?regex .
}'''
for row in graph.query(query):
    print(str(row[0]) + "    " + str(row[1]))

print ("\nknown LogSeries patterns:")
query = '''select ?logseries ?logfmt ?mediatype ?filepattern where {
?logseries a logset:LogSeries .
?logseries logset:logFormatInfo ?filepattern .
?logseries logset:logFormat ?logfmt .
optional { ?logfmt dcat:mediaType ?mediatype } .
filter regex(?filepattern, "^filepattern=") .
}'''
for row in graph.query(query):
    print(row)

regexs for common file patterns:
<dateYYYYMMDD>    (?P<year>20[0-9]{2})(?P<month>0[1-9]|1[0-2])(?P<day>[0-3][0-9])
<timeYYYYMMDDtHHmmss>    (?P<year>20[0-9]{2})(?P<month>0[1-9]|1[0-2])(?P<day>[0-3][0-9])t(?P<hour>[01][0-9]|2[0-3])(?P<minute>[0-5][0-9])(?P<second>[0-5][0-9])
<timeYYYYMMDDHHmmss>    (?P<year>20[0-9]{2})(?P<month>0[1-9]|1[0-2])(?P<day>[0-3][0-9])(?P<hour>[01][0-9]|2[0-3])(?P<minute>[0-5][0-9])(?P<second>[0-5][0-9])

known LogSeries patterns:
(rdflib.term.URIRef('file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict#consumer_logfile'), rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#unstructuredFile'), rdflib.term.Literal('text/plain'), rdflib.term.Literal('filepattern=consumer-<dateYYYYMMDD>'))
(rdflib.term.URIRef('file:///global/homes/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict#consumer_logfile'), rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#unstructure